In [ ]:
# Install Avalanche
!pip install git+https://github.com/ContinualAI/avalanche.git

  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-k9hw_ugf
  Running command git clone --filter=blob:none --quiet https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-k9hw_ugf
  Resolved https://github.com/ContinualAI/avalanche.git to commit 625e46d9203878ed51457f6d7cd8d4e9fb05d093
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.2/134.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.2/585.2 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.6/558.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [ ]:
import avalanche

In [ ]:
avalanche.__version__

'0.6.0a'

##  Split CIFAR-10 using GEM

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.models import SlimResNet18
from avalanche.training.supervised import GEM
from avalanche.evaluation.metrics import (
    forgetting_metrics,
    accuracy_metrics,
    bwt_metrics
)
from avalanche.logging import InteractiveLogger
from avalanche.training.plugins import EvaluationPlugin

In [ ]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Model
model = SlimResNet18(nclasses=10)

In [ ]:
# CL Benchmark Creation
scenario = SplitCIFAR10(n_experiences=5)
train_stream = scenario.train_stream
test_stream = scenario.test_stream

100%|██████████| 170M/170M [00:02<00:00, 76.1MB/s]


Extracting /root/.avalanche/data/cifar10/cifar-10-python.tar.gz to /root/.avalanche/data/cifar10
Files already downloaded and verified


In [ ]:
# Prepare for training & testing
optimizer = SGD(model.parameters(), lr = 0.01, momentum = 0.8, weight_decay=5e-4)
criterion = CrossEntropyLoss()

In [ ]:
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    loggers=[interactive_logger]
)

In [ ]:
# Choose a CL strategy
strategy = GEM(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    patterns_per_exp=2000,  # Increased memory size for better retention
    train_mb_size=256,      # Reduced mini-batch size for stable updates
    train_epochs=15,        # More epochs for thorough task learning
    eval_mb_size=128,       # Keep evaluation batch size manageable
    memory_strength=0.3,    # Stricter gradient projection for retention
    device=device,
    evaluator=eval_plugin
)


In [ ]:
# Train & Test Loop - GEM
for i, train_task in enumerate(train_stream):
    print("\n")
    print("-" * 70)
    print(f"Starting training for Task {i}")
    print("-" * 70)

    strategy.train(train_task)

    print("\n")
    print("-" * 70)
    print(f"Starting evaluation for Task {i}")
    print("-" * 70)

    strategy.eval(test_stream)

    print("\n")



----------------------------------------------------------------------
Starting training for Task 0
----------------------------------------------------------------------
-- >> Start of training phase << --
100%|██████████| 40/40 [01:50<00:00,  2.77s/it]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8450
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9375
100%|██████████| 40/40 [01:47<00:00,  2.69s/it]
Epoch 1 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9379
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
100%|██████████| 40/40 [01:47<00:00,  2.69s/it]
Epoch 2 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9470
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.8125
100%|██████████| 40/40 [01:48<00:00,  2.70s/it]
Epoch 3 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9512
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.8750
100%|██████████| 40/40 [01:47<00:00,  2.68s/it]
Epoch 4 ended.
	Top1_Acc_Epoch/train_ph

KeyboardInterrupt: 